## 1. Configuração do Ambiente

In [ ]:
# ============================================================================
# CONFIGURAÇÃO DO AMBIENTE
# ============================================================================

import warnings
warnings.filterwarnings('ignore')

# Bibliotecas padrão
import json
from pathlib import Path
from datetime import datetime

# Computação científica
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import jarque_bera, shapiro, skew, kurtosis, normaltest

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração de visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 150,
    'font.size': 11,
    'font.family': 'serif',
    'axes.labelsize': 12,
    'axes.titlesize': 14,
})

# Caminhos do projeto
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUTS_DIR = DATA_DIR / 'outputs'
FIGURES_DIR = OUTPUTS_DIR / 'figures'
TABLES_DIR = OUTPUTS_DIR / 'tables'

# Criar diretórios se não existirem
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print("="*60)
print("FASE 4: ANÁLISE ESTATÍSTICA DESCRITIVA")
print("="*60)
print(f"\nData de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Diretório do projeto: {PROJECT_ROOT}")

## 2. Carregamento dos Dados

In [ ]:
# ============================================================================
# CARREGAMENTO DOS DADOS
# ============================================================================

# Carregar retornos processados
returns_path = PROCESSED_DIR / 'returns.csv'
df = pd.read_csv(returns_path, parse_dates=['date'])
df.set_index('date', inplace=True)

print("\n" + "="*60)
print("DADOS CARREGADOS")
print("="*60)

print(f"\n📊 Retornos (returns.csv):")
print(f"   • Período: {df.index.min().strftime('%Y-%m-%d')} a {df.index.max().strftime('%Y-%m-%d')}")
print(f"   • Observações: {len(df):,}")
print(f"   • Colunas: {list(df.columns)}")

# Extrair séries de retornos
r_petr4 = df['r_petr4'].dropna()
r_ibov = df['r_ibov'].dropna()

print(f"\n📈 Séries Extraídas:")
print(f"   • PETR4: {len(r_petr4):,} observações")
print(f"   • IBOV: {len(r_ibov):,} observações")

## 3. Estatísticas Descritivas

In [ ]:
# ============================================================================
# ESTATÍSTICAS DESCRITIVAS
# ============================================================================

def calc_stats(series, name, annualize=252):
    """Calcula estatísticas descritivas completas de uma série."""
    return {
        'name': name,
        'n': len(series),
        # Medidas de posição
        'mean': series.mean(),
        'median': series.median(),
        'min': series.min(),
        'max': series.max(),
        # Quartis
        'q1': series.quantile(0.25),
        'q3': series.quantile(0.75),
        'iqr': series.quantile(0.75) - series.quantile(0.25),
        # Medidas de dispersão
        'std': series.std(),
        'var': series.var(),
        'range': series.max() - series.min(),
        'cv': series.std() / abs(series.mean()) if series.mean() != 0 else np.nan,
        # Medidas de forma
        'skewness': skew(series),
        'kurtosis': kurtosis(series),  # Excesso de curtose (Fisher)
        'kurtosis_raw': kurtosis(series) + 3,  # Curtose bruta
        # Anualizados
        'mean_annual': series.mean() * annualize,
        'std_annual': series.std() * np.sqrt(annualize),
        # Métricas de risco
        'var_95': series.quantile(0.05),  # Value at Risk 95%
        'var_99': series.quantile(0.01),  # Value at Risk 99%
        'cvar_95': series[series <= series.quantile(0.05)].mean(),  # CVaR/ES 95%
    }

# Calcular estatísticas
stats_petr4 = calc_stats(r_petr4, 'PETR4')
stats_ibov = calc_stats(r_ibov, 'IBOVESPA')

print("\n" + "="*70)
print("ESTATÍSTICAS DESCRITIVAS — RETORNOS DIÁRIOS")
print("="*70)

print(f"\n{'Estatística':<25} {'PETR4':>18} {'IBOVESPA':>18}")
print("─"*65)
print(f"{'Observações (n)':<25} {stats_petr4['n']:>18,} {stats_ibov['n']:>18,}")
print("─"*65)
print(f"{'Média (diária)':<25} {stats_petr4['mean']*100:>17.4f}% {stats_ibov['mean']*100:>17.4f}%")
print(f"{'Média (anual)':<25} {stats_petr4['mean_annual']*100:>17.2f}% {stats_ibov['mean_annual']*100:>17.2f}%")
print(f"{'Mediana':<25} {stats_petr4['median']*100:>17.4f}% {stats_ibov['median']*100:>17.4f}%")
print("─"*65)
print(f"{'Desvio Padrão (diário)':<25} {stats_petr4['std']*100:>17.4f}% {stats_ibov['std']*100:>17.4f}%")
print(f"{'Volatilidade (anual)':<25} {stats_petr4['std_annual']*100:>17.2f}% {stats_ibov['std_annual']*100:>17.2f}%")
print(f"{'Coef. de Variação':<25} {stats_petr4['cv']:>18.2f} {stats_ibov['cv']:>18.2f}")
print("─"*65)
print(f"{'Mínimo':<25} {stats_petr4['min']*100:>17.2f}% {stats_ibov['min']*100:>17.2f}%")
print(f"{'1º Quartil (Q1)':<25} {stats_petr4['q1']*100:>17.4f}% {stats_ibov['q1']*100:>17.4f}%")
print(f"{'3º Quartil (Q3)':<25} {stats_petr4['q3']*100:>17.4f}% {stats_ibov['q3']*100:>17.4f}%")
print(f"{'Máximo':<25} {stats_petr4['max']*100:>17.2f}% {stats_ibov['max']*100:>17.2f}%")
print(f"{'Amplitude (Range)':<25} {stats_petr4['range']*100:>17.2f}% {stats_ibov['range']*100:>17.2f}%")
print(f"{'IQR':<25} {stats_petr4['iqr']*100:>17.4f}% {stats_ibov['iqr']*100:>17.4f}%")
print("─"*65)
print(f"{'Assimetria (Skewness)':<25} {stats_petr4['skewness']:>18.4f} {stats_ibov['skewness']:>18.4f}")
print(f"{'Curtose (Excesso)':<25} {stats_petr4['kurtosis']:>18.4f} {stats_ibov['kurtosis']:>18.4f}")
print("─"*65)
print(f"{'VaR 95%':<25} {stats_petr4['var_95']*100:>17.2f}% {stats_ibov['var_95']*100:>17.2f}%")
print(f"{'VaR 99%':<25} {stats_petr4['var_99']*100:>17.2f}% {stats_ibov['var_99']*100:>17.2f}%")
print(f"{'CVaR/ES 95%':<25} {stats_petr4['cvar_95']*100:>17.2f}% {stats_ibov['cvar_95']*100:>17.2f}%")
print("="*65)

## 4. Testes de Normalidade

Testamos a hipótese nula de que os retornos seguem distribuição normal.

In [ ]:
# ============================================================================
# TESTES DE NORMALIDADE
# ============================================================================

def normality_tests(series, name):
    """Executa bateria de testes de normalidade."""
    results = {}
    
    # Jarque-Bera
    jb_stat, jb_pval = jarque_bera(series)
    results['jarque_bera'] = {'statistic': jb_stat, 'p_value': jb_pval}
    
    # Shapiro-Wilk (limitado a 5000 observações)
    sample = series.sample(min(5000, len(series)), random_state=42) if len(series) > 5000 else series
    sw_stat, sw_pval = shapiro(sample)
    results['shapiro_wilk'] = {'statistic': sw_stat, 'p_value': sw_pval}
    
    # D'Agostino-Pearson
    dp_stat, dp_pval = normaltest(series)
    results['dagostino_pearson'] = {'statistic': dp_stat, 'p_value': dp_pval}
    
    return results

# Executar testes
norm_petr4 = normality_tests(r_petr4, 'PETR4')
norm_ibov = normality_tests(r_ibov, 'IBOVESPA')

print("\n" + "="*70)
print("TESTES DE NORMALIDADE")
print("="*70)
print("\nH₀: Os retornos seguem distribuição normal")
print("H₁: Os retornos NÃO seguem distribuição normal")
print("\nNível de significância: α = 0.05")

print(f"\n{'─'*70}")
print(f"{'Teste':<25} {'PETR4':>20} {'IBOVESPA':>20}")
print(f"{'─'*70}")

for test_name in ['jarque_bera', 'shapiro_wilk', 'dagostino_pearson']:
    p1 = norm_petr4[test_name]['p_value']
    p2 = norm_ibov[test_name]['p_value']
    
    name_formatted = test_name.replace('_', '-').title()
    
    sig1 = '***' if p1 < 0.001 else '**' if p1 < 0.01 else '*' if p1 < 0.05 else ''
    sig2 = '***' if p2 < 0.001 else '**' if p2 < 0.01 else '*' if p2 < 0.05 else ''
    
    print(f"{name_formatted:<25} p = {p1:.2e}{sig1:>4} p = {p2:.2e}{sig2:>4}")

print(f"{'─'*70}")
print("*** p < 0.001, ** p < 0.01, * p < 0.05")

# Interpretação
print(f"\n💡 Interpretação:")
reject_petr4 = all(norm_petr4[t]['p_value'] < 0.05 for t in norm_petr4)
reject_ibov = all(norm_ibov[t]['p_value'] < 0.05 for t in norm_ibov)

if reject_petr4:
    print(f"   • PETR4: Rejeitamos H₀ — retornos NÃO são normalmente distribuídos")
else:
    print(f"   • PETR4: Não rejeitamos H₀ — retornos podem ser normais")
    
if reject_ibov:
    print(f"   • IBOV: Rejeitamos H₀ — retornos NÃO são normalmente distribuídos")
else:
    print(f"   • IBOV: Não rejeitamos H₀ — retornos podem ser normais")

print(f"\n   A presença de caudas gordas (kurtosis > 0) e assimetria negativa é")
print(f"   característica típica de retornos financeiros (fato estilizado).")

## 5. Matriz de Correlação

In [ ]:
# ============================================================================
# MATRIZ DE CORRELAÇÃO
# ============================================================================

# Criar DataFrame para correlação
df_corr = df[['r_petr4', 'r_ibov', 'r_petr4_excess', 'r_ibov_excess']].copy()
df_corr.columns = ['PETR4', 'IBOVESPA', 'PETR4 (exc.)', 'IBOV (exc.)']

# Matriz de correlação
corr_matrix = df_corr.corr()

print("\n" + "="*60)
print("MATRIZ DE CORRELAÇÃO")
print("="*60)
print(f"\n{corr_matrix.round(4).to_string()}")

# Correlação principal
corr_main = corr_matrix.loc['PETR4', 'IBOVESPA']

print(f"\n💡 Correlação PETR4 × IBOVESPA: {corr_main:.4f}")
print(f"   R² implícito: {corr_main**2:.4f} ({corr_main**2*100:.2f}%)")

# Interpretação
if corr_main > 0.7:
    print(f"   → Correlação FORTE positiva")
elif corr_main > 0.4:
    print(f"   → Correlação MODERADA positiva")
else:
    print(f"   → Correlação FRACA positiva")

## 6. Persistência dos Resultados

In [ ]:
# ============================================================================
# PERSISTÊNCIA DOS RESULTADOS
# ============================================================================

# Estrutura completa dos resultados
descriptive_results = {
    "sample": {
        "period_start": df.index.min().strftime('%Y-%m-%d'),
        "period_end": df.index.max().strftime('%Y-%m-%d'),
        "n_observations": int(len(df)),
        "frequency": "daily"
    },
    "statistics": {
        "petr4": {k: round(v, 8) if isinstance(v, float) else v 
                  for k, v in stats_petr4.items()},
        "ibov": {k: round(v, 8) if isinstance(v, float) else v 
                 for k, v in stats_ibov.items()}
    },
    "normality_tests": {
        "petr4": {k: {kk: round(vv, 8) for kk, vv in v.items()} 
                  for k, v in norm_petr4.items()},
        "ibov": {k: {kk: round(vv, 8) for kk, vv in v.items()} 
                 for k, v in norm_ibov.items()}
    },
    "correlation": {
        "petr4_ibov": round(corr_main, 6),
        "r_squared_implied": round(corr_main**2, 6)
    },
    "metadata": {
        "generated_at": datetime.now().isoformat(),
        "notebook": "03_descriptive_stats.ipynb",
        "phase": "Fase 4 - Análise Estatística Descritiva"
    }
}

# Salvar JSON
output_path = PROCESSED_DIR / 'descriptive_stats.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(descriptive_results, f, indent=2, ensure_ascii=False)

print("\n" + "="*60)
print("RESULTADOS PERSISTIDOS")
print("="*60)
print(f"\n✅ Arquivo salvo: {output_path}")

## 7. Geração da Tabela LaTeX

In [ ]:
# ============================================================================
# GERAÇÃO DA TABELA LaTeX
# ============================================================================

# Tabela de estatísticas descritivas
latex_table = rf"""\begin{{table}}[htbp]
\centering
\caption{{Estatísticas Descritivas dos Retornos Diários}}
\label{{tab:desc_stats}}
\begin{{tabular}}{{lcc}}
\toprule
\textbf{{Estatística}} & \textbf{{PETR4}} & \textbf{{IBOVESPA}} \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Medidas de Posição}}}} \\
\quad Média (diária) & {stats_petr4['mean']*100:.4f}\% & {stats_ibov['mean']*100:.4f}\% \\
\quad Média (anualizada) & {stats_petr4['mean_annual']*100:.2f}\% & {stats_ibov['mean_annual']*100:.2f}\% \\
\quad Mediana & {stats_petr4['median']*100:.4f}\% & {stats_ibov['median']*100:.4f}\% \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Medidas de Dispersão}}}} \\
\quad Desvio Padrão (diário) & {stats_petr4['std']*100:.4f}\% & {stats_ibov['std']*100:.4f}\% \\
\quad Volatilidade (anual) & {stats_petr4['std_annual']*100:.2f}\% & {stats_ibov['std_annual']*100:.2f}\% \\
\quad Mínimo & {stats_petr4['min']*100:.2f}\% & {stats_ibov['min']*100:.2f}\% \\
\quad Máximo & {stats_petr4['max']*100:.2f}\% & {stats_ibov['max']*100:.2f}\% \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Medidas de Forma}}}} \\
\quad Assimetria & {stats_petr4['skewness']:.4f} & {stats_ibov['skewness']:.4f} \\
\quad Curtose (excesso) & {stats_petr4['kurtosis']:.4f} & {stats_ibov['kurtosis']:.4f} \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Métricas de Risco}}}} \\
\quad VaR 95\% & {stats_petr4['var_95']*100:.2f}\% & {stats_ibov['var_95']*100:.2f}\% \\
\quad CVaR/ES 95\% & {stats_petr4['cvar_95']*100:.2f}\% & {stats_ibov['cvar_95']*100:.2f}\% \\
\midrule
\multicolumn{{3}}{{l}}{{\textit{{Testes de Normalidade (p-valor)}}}} \\
\quad Jarque-Bera & {norm_petr4['jarque_bera']['p_value']:.2e} & {norm_ibov['jarque_bera']['p_value']:.2e} \\
\quad Shapiro-Wilk & {norm_petr4['shapiro_wilk']['p_value']:.2e} & {norm_ibov['shapiro_wilk']['p_value']:.2e} \\
\midrule
Observações (n) & {stats_petr4['n']:,} & {stats_ibov['n']:,} \\
Correlação & \multicolumn{{2}}{{c}}{{{corr_main:.4f}}} \\
\bottomrule
\end{{tabular}}

\vspace{{0.3cm}}
\footnotesize
\textit{{Nota}}: Período: {df.index.min().strftime('%Y-%m-%d')} a {df.index.max().strftime('%Y-%m-%d')}. \\
Retornos logarítmicos diários. Curtose reportada como excesso (Fisher). \\
VaR e CVaR calculados para nível de confiança de 95\%.
\end{{table}}
"""

# Salvar tabela
table_path = TABLES_DIR / 'estatisticas_descritivas.tex'
with open(table_path, 'w', encoding='utf-8') as f:
    f.write(latex_table)

print("\n" + "="*60)
print("TABELA LaTeX GERADA")
print("="*60)
print(f"\n✅ Arquivo salvo: {table_path}")

## 8. Figura: Distribuição dos Retornos

In [ ]:
# ============================================================================
# FIGURA: DISTRIBUIÇÃO DOS RETORNOS
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Cores
color_petr4 = '#2E86AB'  # Azul petróleo
color_ibov = '#A23B72'   # Magenta
color_normal = '#F18F01' # Laranja

for ax, (series, stats, name, color) in zip(
    axes, 
    [(r_petr4, stats_petr4, 'PETR4', color_petr4), 
     (r_ibov, stats_ibov, 'IBOVESPA', color_ibov)]
):
    # Histograma
    n_bins = 80
    ax.hist(series * 100, bins=n_bins, density=True, alpha=0.6, 
            color=color, edgecolor='white', linewidth=0.5,
            label='Histograma')
    
    # KDE
    kde_x = np.linspace(series.min() * 100, series.max() * 100, 500)
    kde = stats.gaussian_kde(series * 100)
    ax.plot(kde_x, kde(kde_x), color=color, linewidth=2.5, 
            label='KDE (densidade empírica)')
    
    # Curva normal teórica
    mu = series.mean() * 100
    sigma = series.std() * 100
    normal_pdf = stats.norm.pdf(kde_x, mu, sigma)
    ax.plot(kde_x, normal_pdf, '--', color=color_normal, linewidth=2,
            label=f'Normal(μ={mu:.2f}, σ={sigma:.2f})')
    
    # Linhas verticais para VaR
    var_95 = series.quantile(0.05) * 100
    ax.axvline(x=var_95, color='red', linestyle=':', linewidth=1.5,
               label=f'VaR 95% = {var_95:.2f}%')
    
    # Formatação
    ax.set_xlabel('Retorno Diário (%)', fontsize=11)
    ax.set_ylabel('Densidade', fontsize=11)
    ax.set_title(f'Distribuição dos Retornos — {name}', fontsize=13, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.set_xlim(series.quantile(0.001) * 100, series.quantile(0.999) * 100)
    
    # Anotação com estatísticas
    stats_text = f"""μ = {series.mean()*100:.4f}%
σ = {series.std()*100:.4f}%
Skew = {skew(series):.2f}
Kurt = {kurtosis(series):.2f}"""
    ax.annotate(stats_text, xy=(0.03, 0.97), xycoords='axes fraction',
                fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.grid(True, alpha=0.3)

plt.tight_layout()

# Salvar figura
fig_path = FIGURES_DIR / 'distribuicao_retornos.pdf'
fig.savefig(fig_path, format='pdf', bbox_inches='tight', dpi=300)

print(f"\n✅ Figura salva: {fig_path}")
plt.show()

## 9. Figura: Heatmap de Correlação

In [ ]:
# ============================================================================
# FIGURA: HEATMAP DE CORRELAÇÃO
# ============================================================================

fig, ax = plt.subplots(figsize=(8, 6))

# Heatmap
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
cmap = sns.diverging_palette(250, 15, s=75, l=40, as_cmap=True, center='light')

sns.heatmap(corr_matrix, 
            mask=mask,
            annot=True, 
            fmt='.4f', 
            cmap=cmap,
            vmin=-1, vmax=1,
            square=True,
            linewidths=0.5,
            cbar_kws={'shrink': 0.8, 'label': 'Correlação'},
            ax=ax)

ax.set_title('Matriz de Correlação — Retornos Diários', fontsize=14, fontweight='bold')

plt.tight_layout()

# Salvar figura
fig_path = FIGURES_DIR / 'correlacao.pdf'
fig.savefig(fig_path, format='pdf', bbox_inches='tight', dpi=300)

print(f"\n✅ Figura salva: {fig_path}")
plt.show()

## 10. Análise Temporal (Rolling Statistics)

In [ ]:
# ============================================================================
# ANÁLISE TEMPORAL — ROLLING STATISTICS
# ============================================================================

# Janela móvel de 252 dias (1 ano)
window = 252

# Calcular rolling statistics
df['rolling_mean_petr4'] = df['r_petr4'].rolling(window=window).mean() * 252  # anualizado
df['rolling_std_petr4'] = df['r_petr4'].rolling(window=window).std() * np.sqrt(252)  # anualizado
df['rolling_mean_ibov'] = df['r_ibov'].rolling(window=window).mean() * 252
df['rolling_std_ibov'] = df['r_ibov'].rolling(window=window).std() * np.sqrt(252)

# Rolling correlation
df['rolling_corr'] = df['r_petr4'].rolling(window=window).corr(df['r_ibov'])

# Figura
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Retorno médio móvel
axes[0].plot(df.index, df['rolling_mean_petr4'] * 100, label='PETR4', color='#2E86AB', linewidth=1.5)
axes[0].plot(df.index, df['rolling_mean_ibov'] * 100, label='IBOVESPA', color='#A23B72', linewidth=1.5)
axes[0].axhline(y=0, color='gray', linestyle='--', linewidth=0.8)
axes[0].set_ylabel('Retorno Anualizado (%)', fontsize=11)
axes[0].set_title(f'Retorno Médio Móvel ({window} dias)', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Volatilidade móvel
axes[1].plot(df.index, df['rolling_std_petr4'] * 100, label='PETR4', color='#2E86AB', linewidth=1.5)
axes[1].plot(df.index, df['rolling_std_ibov'] * 100, label='IBOVESPA', color='#A23B72', linewidth=1.5)
axes[1].set_ylabel('Volatilidade Anual (%)', fontsize=11)
axes[1].set_title(f'Volatilidade Móvel ({window} dias)', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Correlação móvel
axes[2].plot(df.index, df['rolling_corr'], color='#28A745', linewidth=1.5)
axes[2].axhline(y=corr_main, color='red', linestyle='--', linewidth=1, 
                label=f'Correlação média = {corr_main:.3f}')
axes[2].set_ylabel('Correlação', fontsize=11)
axes[2].set_xlabel('Data', fontsize=11)
axes[2].set_title(f'Correlação Móvel PETR4 × IBOV ({window} dias)', fontsize=12, fontweight='bold')
axes[2].legend(loc='lower right')
axes[2].set_ylim(0, 1)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()

# Salvar figura
fig_path = FIGURES_DIR / 'rolling_statistics.pdf'
fig.savefig(fig_path, format='pdf', bbox_inches='tight', dpi=300)

print(f"\n✅ Figura salva: {fig_path}")
plt.show()

## 11. Sumário e Conclusões

In [ ]:
# ============================================================================
# SUMÁRIO FINAL
# ============================================================================

print("\n" + "="*70)
print("SUMÁRIO — ANÁLISE ESTATÍSTICA DESCRITIVA")
print("="*70)

print(f"""
┌─────────────────────────────────────────────────────────────────────┐
│                    RESULTADOS PRINCIPAIS                            │
├─────────────────────────────────────────────────────────────────────┤
│  Período de Análise: {df.index.min().strftime('%Y-%m-%d')} a {df.index.max().strftime('%Y-%m-%d')}                   │
│  Observações: {len(df):,} dias de negociação                            │
├─────────────────────────────────────────────────────────────────────┤
│  RETORNOS                            PETR4         IBOVESPA         │
│    • Média anualizada             {stats_petr4['mean_annual']*100:>7.2f}%        {stats_ibov['mean_annual']*100:>7.2f}%          │
│    • Volatilidade anual           {stats_petr4['std_annual']*100:>7.2f}%        {stats_ibov['std_annual']*100:>7.2f}%          │
│    • Sharpe Ratio (aprox.)        {stats_petr4['mean_annual']/stats_petr4['std_annual']:>7.2f}          {stats_ibov['mean_annual']/stats_ibov['std_annual']:>7.2f}            │
├─────────────────────────────────────────────────────────────────────┤
│  DISTRIBUIÇÃO                                                       │
│    • Assimetria (skewness)        {stats_petr4['skewness']:>7.2f}          {stats_ibov['skewness']:>7.2f}            │
│    • Curtose (excesso)            {stats_petr4['kurtosis']:>7.2f}         {stats_ibov['kurtosis']:>7.2f}           │
│    • Normalidade: {'Rejeitada' if reject_petr4 else 'Não rejeitada':<10}    {'Rejeitada' if reject_ibov else 'Não rejeitada':<10}          │
├─────────────────────────────────────────────────────────────────────┤
│  RISCO                                                              │
│    • VaR 95%                      {stats_petr4['var_95']*100:>7.2f}%        {stats_ibov['var_95']*100:>7.2f}%          │
│    • CVaR/ES 95%                  {stats_petr4['cvar_95']*100:>7.2f}%        {stats_ibov['cvar_95']*100:>7.2f}%          │
├─────────────────────────────────────────────────────────────────────┤
│  CORRELAÇÃO                                                         │
│    • PETR4 × IBOVESPA: {corr_main:.4f}                                      │
│    • R² implícito: {corr_main**2*100:.2f}%                                           │
└─────────────────────────────────────────────────────────────────────┘
""")

print("\n💡 Principais Achados:")
print(f"   1. PETR4 apresenta volatilidade {stats_petr4['std_annual']/stats_ibov['std_annual']:.1f}x maior que o IBOVESPA")
print(f"   2. Ambas as séries exibem caudas gordas (leptocúrticas)")
print(f"   3. Assimetria negativa indica maior frequência de quedas extremas")
print(f"   4. A normalidade é rejeitada — modelos de risco devem considerar isso")

print("\n📁 Artefatos Gerados:")
print(f"   • {PROCESSED_DIR / 'descriptive_stats.json'}")
print(f"   • {TABLES_DIR / 'estatisticas_descritivas.tex'}")
print(f"   • {FIGURES_DIR / 'distribuicao_retornos.pdf'}")
print(f"   • {FIGURES_DIR / 'correlacao.pdf'}")
print(f"   • {FIGURES_DIR / 'rolling_statistics.pdf'}")

print("\n" + "="*70)
print("FASE 4 CONCLUÍDA COM SUCESSO")
print("="*70)